# NOTEBOOK INI ADALAH BAGIAN DIMANA DATASET BERUPA GAMBAR AKAN DIEXPORT MENJADI FILE H5PY

catatan awal:

> untuk file .h5
*     header untuk data 'X' : "x_images"
*     header untuk data 'y' : "y_labels"

> kode label
0. ceplok
1. kawung
2. lereng
3. nitik
4. parang
5. semen
6. lung-lungan

In [1]:
import h5py
import numpy as np
from PIL import Image
import os, os.path, shutil 
import random
from sklearn.utils import shuffle
import imageio
from sklearn.model_selection import train_test_split

# Functions Definition

In [2]:
def download_all_images(folder_dir):
    """ download images from many folders,
        then save image alongside with its label
        
        Parameters:
        ---------------
        folder_dir        String, file location that contain many folders

        Returns:
        ----------
        x_images_array   (N, 224, 224, 3) to be stored, images array, 
        y_images_array    int label code (N, code), associated label
    """
    counter = 0
    x_images_array = []
    y_images_array = []
    for folder_image in os.listdir(folder_dir):
        for filename in os.listdir(f"{folder_dir+folder_image}"):
            img = Image.open(f"{folder_dir+folder_image}/{filename}")
            img = img.convert('RGB')
            img = img.resize((224,224))
            arr_img = np.array(img)
            x_images_array.append(arr_img)
            y_images_array.append(folder_image)
            print(f"working on {folder_image}/{filename}: {np.shape(arr_img)}")
            counter = counter + 1

    print("Done. retrieve " + str(counter) + " datas")
    return x_images_array, y_images_array

In [3]:
def store_many_hdf5(file_location, images, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        file_location     path to HDF5 file, string
        images           images array, (N, 32, 32, 3) to be stored
        labels           labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(f"{file_location}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "x_images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "y_labels", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()
    print(f"{num_images} datas has been stored to HDF5")

In [4]:
def split_train_val(origin_dir, dest_dir):
    """
        split image files into train and validation set
        
        Parameters:
        ---------------
        origin_dir --> origin directory contains whole data in one dataset\n
        dest_dir --> destination directory that will MOVE the VALIDATION DATASET
 
        Returns:
        ----------
        NONE
    """
    for folder_image in os.listdir(origin_dir):
        try: 
            os.mkdir(f"{dest_dir+folder_image}") 
        except OSError as error: 
            print(error)
            
        filenames = random.sample([x for x in os.listdir(f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{folder_image}")], 220) 
        for filename in filenames:
            shutil.move(f"{origin_dir+folder_image}\\{filename}", f"{dest_dir+folder_image}\\{filename}")
        
        print(f"SIZE ORIGIN FOLDER {folder_image}: {str(np.shape(os.listdir(origin_dir+folder_image)))}")
        print(f"SIZE DEST FOLDER {folder_image}: {str(np.shape(os.listdir(dest_dir+folder_image)))}")

# splitting randomly image inside each folder
## 20% for validation dataset
## 80% for training dataset

In [5]:
# this is to split dataset DATASET_ALLFEATURES

"""
==============
WARNING: this code alread executed. 
to re-execute please disable comment
==============
"""
# split_train_val("C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\", "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\validation\\")


'\n==============\nWARNING: this code alread executed. \nto re-execute please disable comment\n==============\n'

# Retrieve Dataset and store to h5py

In [6]:
def create_h5_dataset(source_image, h5_destination, h5_filename):
    """
    assign images into X and y data. then save to h5 file
    
    Parameters:
    source_image -> String, image directory
    h5_destination -> String, h5 destination directory 
    h5_filename -> String, h5 filename
    
    Return:
    None
    """

    print("Working...")

    X, y = download_all_images(source_image)

    # assign y as array
    y = np.array(y).astype(int)

    # SHUFFLE
    X, y = shuffle(X, y, random_state=0)

    # print shape
    print(f"X Shape: {np.shape(X)}\ny Shape: {np.shape(y)}")

    # store dataset
    dest_location = f"{h5_destination}{h5_filename}"
    store_many_hdf5(dest_location, X, y)

    print("DONE")

In [7]:
def create_h5_train_test(source_image, h5_destination, h5_filename):
    """
    assign images into X and y data. then save to h5 file
    
    Parameters:
    source_image -> String, image directory
    h5_destination -> String, h5 destination directory 
    h5_filename -> String, h5 filename
    
    Return:
    None
    """

    print("Working...")

    X, y = download_all_images(source_image)

    # assign y as array
    y = np.array(y).astype(int)

    # SHUFFLE
    X, y = shuffle(X, y, random_state=0)
    
    # split train and test
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

    # print train shape
    print(f"X Train Shape: {np.shape(X_train)}\ny train Shape: {np.shape(y_train)}")
    # print test shape
    print(f"X Val Shape: {np.shape(X_val)}\ny Val Shape: {np.shape(y_val)}")
    
    # store train dataset
    dest_location = f"{h5_destination}train_{h5_filename}"
    store_many_hdf5(dest_location, X_train, y_train)
    # store val dataset
    dest_location = f"{h5_destination}val_{h5_filename}"
    store_many_hdf5(dest_location, X_val, y_val)

    print("DONE")

# MAIN METHOD BELOW

In [8]:
method = "MI"
source = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{method}_dataset\\"
dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
filename = f"{method}_dataset"
create_h5_train_test(source, dest, filename)

Working...
working on 0/0==0531-cl.jpg: (224, 224, 3)
working on 0/0==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/0==ceplok-bintang.jpg: (224, 224, 3)
working on 0/0==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/0==SC015.jpg: (224, 224, 3)
working on 0/0==SC050.jpg: (224, 224, 3)
working on 0/0==YC047.jpg: (224, 224, 3)
working on 0/0==YC086.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Grompol-Grompol-Sogan.jp

working on 0/2==SC050.jpg: (224, 224, 3)
working on 0/2==YC047.jpg: (224, 224, 3)
working on 0/2==YC086.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Grompol-Grompol-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Jamblang--Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sidodrajat Truntum-truntum-,Sogan (1).jpg: (224, 224, 3)
working on 0

working on 0/MI%0==Yogya-Tulis-Ceplok Nanas-Kawung-Sogan.jpg: (224, 224, 3)
working on 0/MI%0==Yogya-Tulis-Ceplok-Truntum,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/MI%0==Yogya-Tulis-Ceplok-Truntum,Cantel-Sogan.jpg: (224, 224, 3)
working on 0/MI%0==Yogya-Tulis-Ceplok-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/MI%0==Yogya-Tulis-Grompol Truntum-Parang,Truntum-Sogan.jpg: (224, 224, 3)
working on 0/MI%0==Yogya-Tulis-Sidomulyo--Sogan.jpg: (224, 224, 3)
working on 0/MI%0==YT055.jpg: (224, 224, 3)
working on 0/MI%0==YT080.jpg: (224, 224, 3)
working on 0/MI%0==YT173.jpg: (224, 224, 3)
working on 0/MI%1==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/MI%1==YC047.jpg: (224, 224, 3)
working on 0/MI%1==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/MI%1==Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/MI%1==Yogya-Tulis-Ceplok Huk Halus-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/MI%1==Yogya-Tulis-Ceplok Keben-Kawung-So

working on 1/1==YC080.jpg: (224, 224, 3)
working on 1/1==YC085.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Banci Kasut-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Bligon melon-Parang,Kawung-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung Benggol Gurdo-Kawung,Gurdo-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung Parang-Kawung,Truntum,Parang-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (3).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (4).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Picis Kembang-Kawung,Kembang-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Tulis-Ceplok Keben-Kawung-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Tulis-Ceplok Nanas-Kawung-Sogan.jpg: (224, 224, 3)
work

working on 1/MI%0==motif-batik-kawung.jpg: (224, 224, 3)
working on 1/MI%0==YC085.jpg: (224, 224, 3)
working on 1/MI%0==Yogya-Cap-Kawung-Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/MI%0==Yogya-Cap-Kawung-Kawung-Sogan (3).jpg: (224, 224, 3)
working on 1/MI%0==Yogya-Tulis-Kasatriyan-Parang,Kawung-Sogan.jpg: (224, 224, 3)
working on 1/MI%0==YT063.jpg: (224, 224, 3)
working on 1/MI%0==YT174.jpg: (224, 224, 3)
working on 1/MI%1==0051-cl.jpg: (224, 224, 3)
working on 1/MI%1==KP006.jpg: (224, 224, 3)
working on 1/MI%1==KP008.jpg: (224, 224, 3)
working on 1/MI%1==KP028.jpg: (224, 224, 3)
working on 1/MI%1==Yogya-Cap-Banci Kasut-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/MI%1==Yogya-Cap-Kawung Benggol Gurdo-Kawung,Gurdo-Sogan.jpg: (224, 224, 3)
working on 1/MI%1==Yogya-Cap-Kawung Parang-Kawung,Truntum,Parang-Sogan (1).jpg: (224, 224, 3)
working on 1/MI%1==Yogya-Cap-Kawung-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/MI%1==Yogya-Tulis-Kawung Ceplok Kawung-Kawung,Ceplok-Sogan.jpg: 

...

working on 4/1==Yogya-Parang-Parang-Sogan (7).jpg: (224, 224, 3)
working on 4/1==Yogya-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/1==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/1==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 4/1==Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/1==Yogya-Tulis-Parang Plenik Seling Sisik-Parang,Sisik,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/2==0068-cl.jpg: (224, 224, 3)
working on 4/2==0170-sample-solo-jateng-btk-s76-L410b.jpg: (224, 224, 3)
working on 4/2==0172-cl.jpg: (224, 224, 3)
working on 4/2==0180-cl.jpg: (224, 224, 3)
working on 4/2==0181-cl.jpg: (224, 224, 3)
working on 4/2==0430-cl.jpg: (224, 224, 3)
working on 4/2==18-Foto-Motif-Batik-Parang-Modern-Terbaru-2019.jpg: (224, 224, 3)
working on 4/2==41e285fdd3250ec139b6440f820d7315.jpg: (224, 224, 3)
working on 4/2==5718057_66380cdd-6950-406d-9f6f-ee55a6ca09ec_850

working on 4/MI%0==motif-batik-parang-rusak2.jpg: (224, 224, 3)
working on 4/MI%0==parangrusakbarong.jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Cap-Parang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Cap-Parang-parang,Gurdo-Sogan (3).jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Cap-Parang-Parang-Sogan (10).jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Cap-Parang-Parang-Sogan (16).jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/MI%0==Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/MI%1==0430-cl.jpg: (224, 224, 3)
working on 4/MI%1==9390097_4bbda498-9c30-45a5-9840-9a0c575628fc.jpg: (224, 224, 3)
working on 4/MI%1==batik-parang.jpg: (224, 224, 3)
working on 4/MI%1==Copy of 0181-cl.jpg: (224, 224, 3)
working on 4/MI%1==JnFi5ah11Y_8.jpg: (224, 224, 3)
working on 4/MI%1==lurik solo lereng.JPG: (224, 224, 3)
working on 4/MI%1==Motif-Parang-Rusak-Sri-Sadoso.jpg: (

working on 5/1==Yogya-Sidomukti Sringit--Sogan (1).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Ceplok Asih-Truntum,Gurdo-Sogan.jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sido mukti--Sogan.jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (1).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (10).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (2).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (3).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (4).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (5).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (6).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (7).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (8).jpg: (224, 224, 3)
working on 5/1==Yogya-Tulis-Sidomukti--Sogan  (9).jpg: (224, 224, 3)
working on 5/2==10.PNG: (224, 224, 3)
working on 5/2==100.PNG: (224, 224, 3)
working on 5/2==10

working on 6/2==122.png: (224, 224, 3)
working on 6/2==13.PNG: (224, 224, 3)
working on 6/2==134.JPG: (224, 224, 3)
working on 6/2==1504a053-s.jpg: (224, 224, 3)
working on 6/2==1504a054.jpg: (224, 224, 3)
working on 6/2==1606a015-s.jpg: (224, 224, 3)
working on 6/2==17f563ef1cff623f73c255e3ac4e9cb6.jpg: (224, 224, 3)
working on 6/2==19.PNG: (224, 224, 3)
working on 6/2==2-1.png: (224, 224, 3)
working on 6/2==2.PNG: (224, 224, 3)
working on 6/2==2019_06_20T11_22_55_07_00.jpg: (224, 224, 3)
working on 6/2==213.jpg: (224, 224, 3)
working on 6/2==2222.jpg: (224, 224, 3)
working on 6/2==3-1.png: (224, 224, 3)
working on 6/2==3.PNG: (224, 224, 3)
working on 6/2==312.JPG: (224, 224, 3)
working on 6/2==3122.JPG: (224, 224, 3)
working on 6/2==3213.JPG: (224, 224, 3)
working on 6/2==33.JPG: (224, 224, 3)
working on 6/2==33.PNG: (224, 224, 3)
working on 6/2==412423.jpg: (224, 224, 3)
working on 6/2==42124.jpg: (224, 224, 3)
working on 6/2==423143243.jpg: (224, 224, 3)
working on 6/2==4312336.JPG

working on 6/MI%2==batik_tulis_lasem_buketan_lunglungan_1433744912_23fe50bf.jpg: (224, 224, 3)
working on 6/MI%2==data.jpeg: (224, 224, 3)
working on 6/MI%2==images.jpg: (224, 224, 3)
working on 6/MI%2==lungan.png: (224, 224, 3)
working on 6/MI%2==lunglungan1.png: (224, 224, 3)
working on 6/MI%3==1-4af35e25d15e65da02ab97a940c03fc2.jpg: (224, 224, 3)
working on 6/MI%3==13.PNG: (224, 224, 3)
working on 6/MI%3==2222.jpg: (224, 224, 3)
working on 6/MI%3==3213.JPG: (224, 224, 3)
working on 6/MI%3==44.PNG: (224, 224, 3)
working on 6/MI%3==441.JPG: (224, 224, 3)
working on 6/MI%3==5082951_9e6f6b7e-5c04-4d83-ad52-15e0de154dbe_770_770.jpg: (224, 224, 3)
working on 6/MI%3==58ca145e98672862ce3c8da15054b2a4.jpg: (224, 224, 3)
working on 6/MI%3==batik_motif_lung_lungan_warna_indigo.jpg: (224, 224, 3)
working on 6/MI%3==IMG_20180515_WA0007_scaled.jpg: (224, 224, 3)
working on 6/MI%3==index.jpg: (224, 224, 3)
working on 6/MI%3==product9a.jpg: (224, 224, 3)
Done. retrieve 2100 datas
X Train Shape: (16

In [8]:
list_method = ["RA", "RO", "FL", "ZO"]
for method in list_method:
    source = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{method}\\Training\\"
    dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
    filename = f"train_geometri_{method}_2200"
    create_h5_dataset(source, dest, filename)

Working...
working on 0/RA%0==0163-cl.jpg: (224, 224, 3)
working on 0/RA%0==0193-cl.jpg: (224, 224, 3)
working on 0/RA%0==batikceplok.jpg: (224, 224, 3)
working on 0/RA%0==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/RA%0==bbr-004-batik-yogya-kombinasi-rakitan-putri-solo.jpg: (224, 224, 3)
working on 0/RA%0==ceplok-bintang.jpg: (224, 224, 3)
working on 0/RA%0==ceplok-bligon.jpg: (224, 224, 3)
working on 0/RA%0==ceplok-ganggong.jpg: (224, 224, 3)
working on 0/RA%0==ceplok1.jpg: (224, 224, 3)
working on 0/RA%0==ceplokkainusa.JPG: (224, 224, 3)
working on 0/RA%0==fadsesad.JPG: (224, 224, 3)
working on 0/RA%0==fdafdase.JPG: (224, 224, 3)
working on 0/RA%0==kotak-kawung-peksi-kreno.jpg: (224, 224, 3)
working on 0/RA%0==KP020.jpg: (224, 224, 3)
working on 0/RA%0==KP021.jpg: (224, 224, 3)
working on 0/RA%0==SC014.jpg: (224, 224, 3)
working on 0/RA%0==SC050.jpg: (224, 224, 3)
working on 0/RA%0==Solo-Cap-Parang-Ceplok Teblem.jpg: (224, 224, 3)
working on 0/RA%0==Truntum-cap-sutra1.jpg:

working on 0/RA%1==Yogya-Tulis-Cantel Jantung-Truntum,Matahari-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Huk Halus-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Huk truntum parang-Parang,Ceplok,Gurdo-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Huk truntum parang-Truntum,Parang-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Keben-Kawung-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Pita-Gurdo,Truntum-Sogan (6).jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Sringit Tumaruntun--Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Tembor-Gurdo-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok Truntum Cantel-Truntum-Sogan (3).jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok-Truntum,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok-Truntum,Cantel-Sogan.jpg: (224, 224, 3)
working on 0/RA%1==Yogya-Tulis-Ceplok-Truntum-Sogan (1).jpg: (22

working on 0/RA%2==Yogya-Tulis-Truntum--Sogan (1).jpg: (224, 224, 3)
working on 0/RA%2==Yogya-Tulis-Truntum--Sogan (2).jpg: (224, 224, 3)
working on 0/RA%2==Yogya-Tulis-Truntum-Ukel-Sogan.jpg: (224, 224, 3)
working on 0/RA%2==Yogya-Tulis-Ukel Ceplok Bintang-Ukel,Bintang,Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/RA%2==YT027.jpg: (224, 224, 3)
working on 0/RA%2==YT037.jpg: (224, 224, 3)
working on 0/RA%2==YT038.jpg: (224, 224, 3)
working on 0/RA%2==YT055.jpg: (224, 224, 3)
working on 0/RA%2==YT056.jpg: (224, 224, 3)
working on 0/RA%2==YT065.jpg: (224, 224, 3)
working on 0/RA%2==YT068.jpg: (224, 224, 3)
working on 0/RA%2==YT080.jpg: (224, 224, 3)
working on 0/RA%2==YT173.jpg: (224, 224, 3)
working on 0/RA%3==0163-cl.jpg: (224, 224, 3)
working on 0/RA%3==0193-cl.jpg: (224, 224, 3)
working on 0/RA%3==batikceplok.jpg: (224, 224, 3)
working on 0/RA%3==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/RA%3==bbr-004-batik-yogya-kombinasi-rakitan-putri-solo.jpg: (224, 224, 3)
working on 0

working on 0/RAori==Yogya-Cap-Sidomulyo--Sogan (3).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Sidomulyo--Sogan (4).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Tambal-Parang,Truntum-Sogan.jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (1).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Truntum Gurdo-Truntum,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Truntum Gurdo-Truntum,Gurdo-Sogan (3).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Truntum-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Cap-Truntum-Truntum-Sogan (2).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Grompol--Sogan (1).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Grompol--Sogan (2).jpg: (224, 224, 3)
working on 0/RAori==Yogya-Tulis-Cantel Jantung-Truntum,Matahari-Sogan.jpg: (224, 224, 3)
working on 0/RAori==Yogya-Tulis-Ceplok Huk Halus-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/RAori==Yogya-Tulis-Ceplok Huk truntum parang-Parang,Ceplok,

working on 1/RA%0==Yogya-Tulis-Kawung Bolem-Kawung-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==Yogya-Tulis-Kawung Ceplok Kawung-Kawung,Ceplok-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==Yogya-Tulis-Kotak Bintang-Bintang,Kawung-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==Yogya-Tulis-Picis Ceplok Huk-Kawung,Gurdo,Ceplok-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==Yogya-Tulis-Picis Gurdo-Kawung,Gurdo-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==Yogya-Tulis-Sri rejeki-Kawung-Sogan.jpg: (224, 224, 3)
working on 1/RA%0==YT063.jpg: (224, 224, 3)
working on 1/RA%0==YT064.jpg: (224, 224, 3)
working on 1/RA%0==YT072.jpg: (224, 224, 3)
working on 1/RA%0==YT076.jpg: (224, 224, 3)
working on 1/RA%0==YT083.jpg: (224, 224, 3)
working on 1/RA%0==YT098.jpg: (224, 224, 3)
working on 1/RA%0==YT109.jpg: (224, 224, 3)
working on 1/RA%1==0051-cl.jpg: (224, 224, 3)
working on 1/RA%1==0051-kp-mdr-jatim-btk-s76-L42a.jpg: (224, 224, 3)
working on 1/RA%1==0167-sample-solo-jateng-btk-s76-L410b.jpg: (224, 224, 3)
w

...

working on 3/ZO%3==Yogya-Nitik-Nitik,Sekar-Sogan (9).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (1).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (14).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (15).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (17).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (18).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (2).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (4).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (5).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (7).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Nitik-Nitik-Sogan (8).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Tulis-Kokrosono--Sogan (1).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Tulis-Kokrosono--Sogan (2).jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Tulis-Nitik Jongrong-Nitik-Sogan.jpg: (224, 224, 3)
working on 3/ZO%3==Yogya-Tulis-Prabu anom Tru

working on 4/ZO%0==parang-nitik.jpg: (224, 224, 3)
working on 4/ZO%0==parang-rusak (1).jpg: (224, 224, 3)
working on 4/ZO%0==parang-rusak-classic-geometric-pattern-of-javanese-batik-motif-FD41NP.jpg: (224, 224, 3)
working on 4/ZO%0==Parang-Soblog.jpg: (224, 224, 3)
working on 4/ZO%0==parangbarislatarcemeng.jpg: (224, 224, 3)
working on 4/ZO%0==parangrusakbarong.jpg: (224, 224, 3)
working on 4/ZO%0==SC071.jpg: (224, 224, 3)
working on 4/ZO%0==SC097.jpg: (224, 224, 3)
working on 4/ZO%0==SC098.jpg: (224, 224, 3)
working on 4/ZO%0==YC065.jpg: (224, 224, 3)
working on 4/ZO%0==YC069.jpg: (224, 224, 3)
working on 4/ZO%0==YK056.jpg: (224, 224, 3)
working on 4/ZO%0==YK059.jpg: (224, 224, 3)
working on 4/ZO%0==YK065.jpg: (224, 224, 3)
working on 4/ZO%0==Yogya-Cap-Parang Keong-Parang,Keong-Sogan (1).jpg: (224, 224, 3)
working on 4/ZO%0==Yogya-Cap-Parang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 4/ZO%0==Yogya-Cap-Parang-parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/ZO%0==Yogya-Ca

working on 4/ZO%1==Yogya-Parang-Parang-Sogan (13).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Parang-Parang-Sogan (16).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Parang-Parang-Sogan (2).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Parang-Parang-Sogan (3).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Parang-Parang-Sogan (4).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Tulis-Mendut-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Tulis-Parang Nitik-Nitik-Sogan.jpg: (224, 224, 3)
working on 4/ZO%1==Yogya-Tulis-Parang Plenik Seling Sisik-Parang,Sisik,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/ZO%2==0068-cl.jpg: (224, 224, 3)
working on 4/ZO%2==0093-cl.jpg: (224, 224, 3)
working on 4/ZO%2==0146-cl.jpg: (224, 224, 3)
working on 

working on 4/ZO%3==parangbarislatarcemeng.jpg: (224, 224, 3)
working on 4/ZO%3==parangrusakbarong.jpg: (224, 224, 3)
working on 4/ZO%3==SC071.jpg: (224, 224, 3)
working on 4/ZO%3==SC097.jpg: (224, 224, 3)
working on 4/ZO%3==SC098.jpg: (224, 224, 3)
working on 4/ZO%3==YC065.jpg: (224, 224, 3)
working on 4/ZO%3==YC069.jpg: (224, 224, 3)
working on 4/ZO%3==YK056.jpg: (224, 224, 3)
working on 4/ZO%3==YK059.jpg: (224, 224, 3)
working on 4/ZO%3==YK065.jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang Keong-Parang,Keong-Sogan (1).jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-parang,Gurdo-Sogan (3).jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-Parang-Sogan (10).jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-Parang-Sogan (11).jpg: (224, 224, 3)
working on 4/ZO%3==Yogya-Cap-Parang-Parang-Sogan (12).jpg: (224, 224,

working on 4/ZOori==Yogya-Parang-Parang-Sogan (3).jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Parang-Parang-Sogan (4).jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Tulis-Mendut-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Tulis-Parang Nitik-Nitik-Sogan.jpg: (224, 224, 3)
working on 4/ZOori==Yogya-Tulis-Parang Plenik Seling Sisik-Parang,Sisik,Gurdo-Sogan.jpg: (224, 224, 3)
Done. retrieve 2200 datas
X Shape: (2200, 224, 224, 3)
y Shape: (2200,)
2200 datas has been stored to HDF5
DONE


In [8]:
source = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\testing_dataset_gray\\"
dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
filename = "testing_dataset_gray"
create_h5_dataset(source, dest, filename)

Working...
working on 0/SL%0%==0531-cl.jpg: (224, 224, 3)
working on 0/SL%0%==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%0%==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%0%==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%0%==SC015.jpg: (224, 224, 3)
working on 0/SL%0%==SC050.jpg: (224, 224, 3)
working on 0/SL%0%==YC047.jpg: (224, 224, 3)
working on 0/SL%0%==YC086.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/SL%0%==Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224

working on 0/SL%1%==Yogya-Tulis-Trimino-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%1%==YT037.jpg: (224, 224, 3)
working on 0/SL%1%==YT038.jpg: (224, 224, 3)
working on 0/SL%1%==YT055.jpg: (224, 224, 3)
working on 0/SL%1%==YT056.jpg: (224, 224, 3)
working on 0/SL%1%==YT065.jpg: (224, 224, 3)
working on 0/SL%1%==YT080.jpg: (224, 224, 3)
working on 0/SL%1%==YT117.jpg: (224, 224, 3)
working on 0/SL%1%==YT173.jpg: (224, 224, 3)
working on 0/SL%2%==0531-cl.jpg: (224, 224, 3)
working on 0/SL%2%==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%2%==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%2%==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%2%==SC015.jpg: (224, 224, 3)
working on 0/SL%2%==SC050.jpg: (224, 224, 3)
working on 0/SL%2%==YC047.jpg: (224, 224, 3)
working on 0/SL%2%==YC086.jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%2%==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on

working on 0/SL%3%==YT080.jpg: (224, 224, 3)
working on 0/SL%3%==YT117.jpg: (224, 224, 3)
working on 0/SL%3%==YT173.jpg: (224, 224, 3)
working on 0/SL%FL==0531-cl.jpg: (224, 224, 3)
working on 0/SL%FL==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%FL==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%FL==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%FL==SC015.jpg: (224, 224, 3)
working on 0/SL%FL==SC050.jpg: (224, 224, 3)
working on 0/SL%FL==YC047.jpg: (224, 224, 3)
working on 0/SL%FL==YC086.jpg: (224, 224, 3)
working on 0/SL%FL==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%FL==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/SL%FL==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/SL%FL==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/SL%FL==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/SL%

working on 0/SL%MI%==YT055.jpg: (224, 224, 3)
working on 0/SL%MI%==YT056.jpg: (224, 224, 3)
working on 0/SL%MI%==YT065.jpg: (224, 224, 3)
working on 0/SL%MI%==YT080.jpg: (224, 224, 3)
working on 0/SL%MI%==YT117.jpg: (224, 224, 3)
working on 0/SL%MI%==YT173.jpg: (224, 224, 3)
working on 0/SL%RA==0531-cl.jpg: (224, 224, 3)
working on 0/SL%RA==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%RA==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%RA==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%RA==SC015.jpg: (224, 224, 3)
working on 0/SL%RA==SC050.jpg: (224, 224, 3)
working on 0/SL%RA==YC047.jpg: (224, 224, 3)
working on 0/SL%RA==YC086.jpg: (224, 224, 3)
working on 0/SL%RA==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%RA==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/SL%RA==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/SL%RA==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahar

working on 0/SL%RO==Yogya-Tulis-Trimino-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/SL%RO==YT037.jpg: (224, 224, 3)
working on 0/SL%RO==YT038.jpg: (224, 224, 3)
working on 0/SL%RO==YT055.jpg: (224, 224, 3)
working on 0/SL%RO==YT056.jpg: (224, 224, 3)
working on 0/SL%RO==YT065.jpg: (224, 224, 3)
working on 0/SL%RO==YT080.jpg: (224, 224, 3)
working on 0/SL%RO==YT117.jpg: (224, 224, 3)
working on 0/SL%RO==YT173.jpg: (224, 224, 3)
working on 0/SL%ZO==0531-cl.jpg: (224, 224, 3)
working on 0/SL%ZO==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/SL%ZO==ceplok-bintang.jpg: (224, 224, 3)
working on 0/SL%ZO==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/SL%ZO==SC015.jpg: (224, 224, 3)
working on 0/SL%ZO==SC050.jpg: (224, 224, 3)
working on 0/SL%ZO==YC047.jpg: (224, 224, 3)
working on 0/SL%ZO==YC086.jpg: (224, 224, 3)
working on 0/SL%ZO==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/SL%ZO==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on

...

working on 5/SL%MI%==Capture.PNG: (224, 224, 3)
working on 5/SL%MI%==ce67c7ea0dbb133fbd59963959afc8b4.jpg: (224, 224, 3)
working on 5/SL%MI%==lawasan-1.jpg: (224, 224, 3)
working on 5/SL%MI%==SEMEN.jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Cap-Sidomukti Cemeng-Gurdo-Sogan.jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Cap-Sidomukti Sringit--Sogan.jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Sidomukti Sringit--Sogan (1).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Ceplok Asih-Truntum,Gurdo-Sogan.jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sido mukti--Sogan.jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sidomukti--Sogan  (1).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sidomukti--Sogan  (10).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sidomukti--Sogan  (2).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sidomukti--Sogan  (3).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tulis-Sidomukti--Sogan  (4).jpg: (224, 224, 3)
working on 5/SL%MI%==Yogya-Tu

working on 5/SL%ZO==109.PNG: (224, 224, 3)
working on 5/SL%ZO==11.PNG: (224, 224, 3)
working on 5/SL%ZO==111.PNG: (224, 224, 3)
working on 5/SL%ZO==122.PNG: (224, 224, 3)
working on 5/SL%ZO==1222.PNG: (224, 224, 3)
working on 5/SL%ZO==123.PNG: (224, 224, 3)
working on 5/SL%ZO==124.PNG: (224, 224, 3)
working on 5/SL%ZO==13.PNG: (224, 224, 3)
working on 5/SL%ZO==14.PNG: (224, 224, 3)
working on 5/SL%ZO==2.PNG: (224, 224, 3)
working on 5/SL%ZO==21210307_64436331-fb8c-4866-9984-2eac675f4c93.jpg: (224, 224, 3)
working on 5/SL%ZO==234.PNG: (224, 224, 3)
working on 5/SL%ZO==244.PNG: (224, 224, 3)
working on 5/SL%ZO==314.PNG: (224, 224, 3)
working on 5/SL%ZO==345.PNG: (224, 224, 3)
working on 5/SL%ZO==4.PNG: (224, 224, 3)
working on 5/SL%ZO==43.PNG: (224, 224, 3)
working on 5/SL%ZO==44.PNG: (224, 224, 3)
working on 5/SL%ZO==444.PNG: (224, 224, 3)
working on 5/SL%ZO==45.PNG: (224, 224, 3)
working on 5/SL%ZO==450.PNG: (224, 224, 3)
working on 5/SL%ZO==476.PNG: (224, 224, 3)
working on 5/SL%ZO==4

working on 6/SL%1%==6-1.png: (224, 224, 3)
working on 6/SL%1%==6.PNG: (224, 224, 3)
working on 6/SL%1%==79941f2d103228345ec8b15d3e6331ab.jpg: (224, 224, 3)
working on 6/SL%1%==898aa5a9d3f54902296c451156a3e597.jpg: (224, 224, 3)
working on 6/SL%1%==9984.JPG: (224, 224, 3)
working on 6/SL%1%==batik-sida-mukti-men_batik-sida-mukti-men-hemd-katun-panjang-hitam-colet-warna-motif-lung-lungan-kembang_full04.jpg: (224, 224, 3)
working on 6/SL%1%==Batik_gringsing_motif_lung_lungan.JPG: (224, 224, 3)
working on 6/SL%1%==batik_motif_lung_lungan_warna_indigo.jpg: (224, 224, 3)
working on 6/SL%1%==batik_tulis_lasem_buketan_lunglungan_1433744912_23fe50bf.jpg: (224, 224, 3)
working on 6/SL%1%==batik_tulis_lasem_buketan_lunglungan_1433744913_32515ca5.jpg: (224, 224, 3)
working on 6/SL%1%==Batik_Tulis_Lasem_Motif_Sekar_Jagat_Lung_lungan.jpg: (224, 224, 3)
working on 6/SL%1%==Capture.PNG: (224, 224, 3)
working on 6/SL%1%==data.jpeg: (224, 224, 3)
working on 6/SL%1%==data.png: (224, 224, 3)
working on 6/

working on 6/SL%FL==13.PNG: (224, 224, 3)
working on 6/SL%FL==134.JPG: (224, 224, 3)
working on 6/SL%FL==1504a053-s.jpg: (224, 224, 3)
working on 6/SL%FL==1504a054.jpg: (224, 224, 3)
working on 6/SL%FL==1606a015-s.jpg: (224, 224, 3)
working on 6/SL%FL==17f563ef1cff623f73c255e3ac4e9cb6.jpg: (224, 224, 3)
working on 6/SL%FL==19.PNG: (224, 224, 3)
working on 6/SL%FL==2-1.png: (224, 224, 3)
working on 6/SL%FL==2.PNG: (224, 224, 3)
working on 6/SL%FL==2019_06_20T11_22_55_07_00.jpg: (224, 224, 3)
working on 6/SL%FL==213.jpg: (224, 224, 3)
working on 6/SL%FL==2222.jpg: (224, 224, 3)
working on 6/SL%FL==3-1.png: (224, 224, 3)
working on 6/SL%FL==3.PNG: (224, 224, 3)
working on 6/SL%FL==312.JPG: (224, 224, 3)
working on 6/SL%FL==3122.JPG: (224, 224, 3)
working on 6/SL%FL==3213.JPG: (224, 224, 3)
working on 6/SL%FL==33.JPG: (224, 224, 3)
working on 6/SL%FL==33.PNG: (224, 224, 3)
working on 6/SL%FL==412423.jpg: (224, 224, 3)
working on 6/SL%FL==42124.jpg: (224, 224, 3)
working on 6/SL%FL==4231432

working on 6/SL%RA==9984.JPG: (224, 224, 3)
working on 6/SL%RA==batik-sida-mukti-men_batik-sida-mukti-men-hemd-katun-panjang-hitam-colet-warna-motif-lung-lungan-kembang_full04.jpg: (224, 224, 3)
working on 6/SL%RA==Batik_gringsing_motif_lung_lungan.JPG: (224, 224, 3)
working on 6/SL%RA==batik_motif_lung_lungan_warna_indigo.jpg: (224, 224, 3)
working on 6/SL%RA==batik_tulis_lasem_buketan_lunglungan_1433744912_23fe50bf.jpg: (224, 224, 3)
working on 6/SL%RA==batik_tulis_lasem_buketan_lunglungan_1433744913_32515ca5.jpg: (224, 224, 3)
working on 6/SL%RA==Batik_Tulis_Lasem_Motif_Sekar_Jagat_Lung_lungan.jpg: (224, 224, 3)
working on 6/SL%RA==Capture.PNG: (224, 224, 3)
working on 6/SL%RA==data.jpeg: (224, 224, 3)
working on 6/SL%RA==data.png: (224, 224, 3)
working on 6/SL%RA==images.jpg: (224, 224, 3)
working on 6/SL%RA==IMG_20180515_WA0007_scaled.jpg: (224, 224, 3)
working on 6/SL%RA==IMG_20190621_WA0064.jpg: (224, 224, 3)
working on 6/SL%RA==index.jpg: (224, 224, 3)
working on 6/SL%RA==lung-

In [8]:
# source = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\Dataset_Training2\\"
# dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
# filename = "allfeature_dataset"
# create_h5_train_test(source, dest, filename)

# source = "C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\noaugm_dataset\\"
# dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
# filename = "noaugm_dataset"
# create_h5_train_test(source, dest, filename)

list_method = ["ZO", "FL"]
for method in list_method:
    source = f"C:\\SAMUEL\\KULIAH\\Skripsi Gasal 1920\\jupyter-notebook\\new\\{method}_dataset\\"
    dest = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/"
    filename = f"{method}_dataset"
    create_h5_train_test(source, dest, filename)

Working...
working on 0/0==0531-cl.jpg: (224, 224, 3)
working on 0/0==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/0==ceplok-bintang.jpg: (224, 224, 3)
working on 0/0==ceplok-bligon1.jpg: (224, 224, 3)
working on 0/0==SC015.jpg: (224, 224, 3)
working on 0/0==SC050.jpg: (224, 224, 3)
working on 0/0==YC047.jpg: (224, 224, 3)
working on 0/0==YC086.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Cabut Solo-Truntum-Sogan (1).jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Kawung-Ceplok,Kawung-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224, 224, 3)
working on 0/0==Yogya-Cap-Grompol-Grompol-Sogan.jp

working on 0/2==Yogya-Cap-Ceplok Kembang Jeruk-Truntum, Ceplok,Parang-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Matahari-Ceplok, Parang,Matahari-Sogan - Copy.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok Sisik Bintang-Parang,Bintang-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Ceplok-Ceplok-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Grompol kecil-Grompol-Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Grompol-Grompol-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Jamblang--Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sawitan Pintu Retno-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sidodrajat Truntum-truntum-,Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sidomulyo--Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Sidomulyo--Sogan (4).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (1).jpg: (224, 224, 3)
working on 0/2==Yogya-Cap-Truntum Gurdo-Truntum,Gurdo-Sogan (1).jpg: (22

working on 0/ZO%0==Yogya-Tulis-Ceplok Huk Halus-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Ceplok-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Keteblem Gebel-Truntum-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Kupu manis-Truntum,Kupu-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Prabu Anom Truntum-Truntum,Gurdo-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Sekar kedaton-Parang,Truntum-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==Yogya-Tulis-Tambal-Parang,Kawung,Truntum-Sogan.jpg: (224, 224, 3)
working on 0/ZO%0==YT056.jpg: (224, 224, 3)
working on 0/ZO%0==YT173.jpg: (224, 224, 3)
working on 0/ZO%1==batik_Grompol_motif_w.jpg: (224, 224, 3)
working on 0/ZO%1==ceplok-bintang.jpg: (224, 224, 3)
working on 0/ZO%1==SC050.jpg: (224, 224, 3)
working on 0/ZO%1==Yogya-Cap-Jamblang--Sogan.jpg: (224, 224, 3)
working on 0/ZO%1==Yogya-Cap-Sidodrajat Truntum-truntum-,Sogan (1).jpg: (224, 224, 3)
working on 0/ZO%1==Yogya-Cap-Truntu

working on 1/1==YC080.jpg: (224, 224, 3)
working on 1/1==YC085.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Banci Kasut-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Bligon melon-Parang,Kawung-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung Benggol Gurdo-Kawung,Gurdo-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung Parang-Kawung,Truntum,Parang-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (2).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (3).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Kawung-Kawung-Sogan (4).jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Picis Kembang-Kawung,Kembang-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/1==Yogya-Tulis-Ceplok Keben-Kawung-Sogan.jpg: (224, 224, 3)
working on 1/1==Yogya-Tulis-Ceplok Nanas-Kawung-Sogan.jpg: (224, 224, 3)
work

working on 1/ZO%0==KP005.jpg: (224, 224, 3)
working on 1/ZO%0==KP007.jpg: (224, 224, 3)
working on 1/ZO%0==KP026.jpg: (224, 224, 3)
working on 1/ZO%0==KP027.jpg: (224, 224, 3)
working on 1/ZO%0==KP028.jpg: (224, 224, 3)
working on 1/ZO%0==KP031.jpg: (224, 224, 3)
working on 1/ZO%0==kumbang-sogan-coklat2-09.jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Cap-Kawung-Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Cap-Kawung-Kawung-Sogan (4).jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Cap-Tambal-Truntum,Parang,Kawung-Sogan (1).jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Tulis-Kasatriyan-Parang,Kawung-Sogan.jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Tulis-Kawung Bolem-Kawung-Sogan.jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Tulis-Kawung Ceplok Kawung-Kawung,Ceplok-Sogan.jpg: (224, 224, 3)
working on 1/ZO%0==Yogya-Tulis-Picis Ceplok Huk-Kawung,Gurdo,Ceplok-Sogan.jpg: (224, 224, 3)
working on 1/ZO%0==YT063.jpg: (224, 224, 3)
working on 1/ZO%1==0051-cl.jpg: (224, 224, 3)
working on 

...

working on 4/2==Yogya-Cap-Parang-Parang-Sogan (25).jpg: (224, 224, 3)
working on 4/2==Yogya-Cap-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang (17)-Sogan.jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (10).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (11).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (13).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (3).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (4).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (7).jpg: (224, 224, 3)
working on 4/2==Yogya-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/2==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/2==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 4/2==Yogya-Tulis-Parang Klithik Gurdo-parang,Gurdo-Sogan.jpg: (224, 224, 3)
working on 4/2==Yogya-Tulis-Parang Plenik Seling Sisik-Parang,Sisik,

working on 4/FL%3==Yogya-Cap-Parang-Parang-Sogan (9).jpg: (224, 224, 3)
working on 4/FL%3==Yogya-Parang-Parang (17)-Sogan.jpg: (224, 224, 3)
working on 4/FL%3==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (1).jpg: (224, 224, 3)
working on 4/FL%3==Yogya-Tulis-Parang Gendreh Gurdo-Parang,Gurdo-Sogan (2).jpg: (224, 224, 3)
working on 5/0==10.PNG: (224, 224, 3)
working on 5/0==100.PNG: (224, 224, 3)
working on 5/0==102.PNG: (224, 224, 3)
working on 5/0==1023.PNG: (224, 224, 3)
working on 5/0==109.PNG: (224, 224, 3)
working on 5/0==11.PNG: (224, 224, 3)
working on 5/0==111.PNG: (224, 224, 3)
working on 5/0==122.PNG: (224, 224, 3)
working on 5/0==1222.PNG: (224, 224, 3)
working on 5/0==123.PNG: (224, 224, 3)
working on 5/0==124.PNG: (224, 224, 3)
working on 5/0==13.PNG: (224, 224, 3)
working on 5/0==14.PNG: (224, 224, 3)
working on 5/0==2.PNG: (224, 224, 3)
working on 5/0==21210307_64436331-fb8c-4866-9984-2eac675f4c93.jpg: (224, 224, 3)
working on 5/0==234.PNG: (224, 224, 3)
working o

working on 5/2==lawasan-1.jpg: (224, 224, 3)
working on 5/2==SEMEN.jpg: (224, 224, 3)
working on 5/2==Yogya-Cap-Sidomukti Cemeng-Gurdo-Sogan.jpg: (224, 224, 3)
working on 5/2==Yogya-Cap-Sidomukti Sringit--Sogan.jpg: (224, 224, 3)
working on 5/2==Yogya-Sidomukti Sringit--Sogan (1).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Ceplok Asih-Truntum,Gurdo-Sogan.jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sido mukti--Sogan.jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (1).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (10).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (2).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (3).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (4).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (5).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (6).jpg: (224, 224, 3)
working on 5/2==Yogya-Tulis-Sidomukti--Sogan  (7).jpg: (224, 224, 3)
wor

working on 6/0==2-1.png: (224, 224, 3)
working on 6/0==2.PNG: (224, 224, 3)
working on 6/0==2019_06_20T11_22_55_07_00.jpg: (224, 224, 3)
working on 6/0==213.jpg: (224, 224, 3)
working on 6/0==2222.jpg: (224, 224, 3)
working on 6/0==3-1.png: (224, 224, 3)
working on 6/0==3.PNG: (224, 224, 3)
working on 6/0==312.JPG: (224, 224, 3)
working on 6/0==3122.JPG: (224, 224, 3)
working on 6/0==3213.JPG: (224, 224, 3)
working on 6/0==33.JPG: (224, 224, 3)
working on 6/0==33.PNG: (224, 224, 3)
working on 6/0==412423.jpg: (224, 224, 3)
working on 6/0==42124.jpg: (224, 224, 3)
working on 6/0==423143243.jpg: (224, 224, 3)
working on 6/0==4312336.JPG: (224, 224, 3)
working on 6/0==4323222.jpeg: (224, 224, 3)
working on 6/0==44.PNG: (224, 224, 3)
working on 6/0==441.JPG: (224, 224, 3)
working on 6/0==4af35e25d15e65da02ab97a940c03fc2.jpg: (224, 224, 3)
working on 6/0==5-1.png: (224, 224, 3)
working on 6/0==5.PNG: (224, 224, 3)
working on 6/0==5082951_9e6f6b7e-5c04-4d83-ad52-15e0de154dbe_770_770.jpg: (22

working on 6/2==IMG_20190621_WA0064.jpg: (224, 224, 3)
working on 6/2==index.jpg: (224, 224, 3)
working on 6/2==lung-lungan-ungu-latar-coklat.jpg: (224, 224, 3)
working on 6/2==lungan.png: (224, 224, 3)
working on 6/2==lunglungan1.png: (224, 224, 3)
working on 6/2==lunglungan2.png: (224, 224, 3)
working on 6/2==product9a.jpg: (224, 224, 3)
working on 6/2==SU_2086_BK__1_.JPG: (224, 224, 3)
working on 6/3==0863-kp-btg-jateng-btk-b87-L32b.jpg: (224, 224, 3)
working on 6/3==1-4af35e25d15e65da02ab97a940c03fc2.jpg: (224, 224, 3)
working on 6/3==12-1.png: (224, 224, 3)
working on 6/3==12.PNG: (224, 224, 3)
working on 6/3==122.png: (224, 224, 3)
working on 6/3==13.PNG: (224, 224, 3)
working on 6/3==134.JPG: (224, 224, 3)
working on 6/3==1504a053-s.jpg: (224, 224, 3)
working on 6/3==1504a054.jpg: (224, 224, 3)
working on 6/3==1606a015-s.jpg: (224, 224, 3)
working on 6/3==17f563ef1cff623f73c255e3ac4e9cb6.jpg: (224, 224, 3)
working on 6/3==19.PNG: (224, 224, 3)
working on 6/3==2-1.png: (224, 224,

# Retrieve Dataset

In [ ]:
print("Working...")

X, y = download_all_images("C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/validation/")

# assign y as array
y = np.array(y).astype(int)

# SHUFFLE
X, y = shuffle(X, y, random_state=0)

# print shape
print(f"X Shape: {np.shape(X)}\ny Shape: {np.shape(y)}")



# Store to HDF5

In [8]:
dest_location = "C:/SAMUEL/KULIAH/Skripsi Gasal 1920/jupyter-notebook/dataset_batik/h5_file/ALLFEATURES_NONSPLIT"
store_many_hdf5(dest_location, X, y)

print("DONE")

stored to HDF5
DONE
